In [2]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from haversine import haversine

In [8]:
#Соединяем запаршенные датасеты
df12 = pd.read_csv('dataset4_0.csv', sep = ';', header=None)
df13 = pd.read_csv('dataset4_1.csv', sep = ';', header=None)
df14 = pd.read_csv('dataset4_2.csv', sep = ';', header=None)
df15 = pd.read_csv('dataset4_3.csv', sep = ';', header=None)
df16 = pd.read_csv('dataset4_4.csv', sep = ';', header=None)
df17 = pd.read_csv('dataset4_5.csv', sep = ';', header=None)
df18 = pd.read_csv('dataset4_33.csv', sep = ';', header=None)
columns2 = ['Metro station', 'lon', 'lat', 'Number of floors', 'type', 'year', 'Kitchen area', 'Number of rooms',  'id', 'Area', 'flat', 'floor', 'Living area', 'Price', 'date']

df12.columns = columns2
df13.columns = columns2
df14.columns = columns2
df15.columns = columns2
df16.columns = columns2
df17.columns = columns2
df18.columns = columns2

df = pd.concat([df12, df13, df14, df15, df16, df17], ignore_index = True)
#Удалим ненужные фичи
df = df.drop(columns = ['year', 'flat'])
df = df.dropna(subset = 'Price')
df = df[(df.type != 'wood')&(df.type != 'aerocreteBlock')]

#заполним Модой
df.type = df.type.fillna('monolith')

#Приведем в нижний регистр и удалим пробелы
df['Metro station'] = df['Metro station'].str.lower()
df['Metro station'] = df['Metro station'].str.replace(' ', '')

# там где чисто комнат Nan - квартира стулия, заменим на 0
df['Number of rooms'].fillna(0, inplace = True)

# содержат много Nan
df.drop(columns = ['Kitchen area', 'Living area'], inplace = True)

# Удалим дубликатные квартиры
df = df.drop_duplicates(subset = 'id')

# Оставим средний ценовой сегмент (По усам боксплота)
df = df[df.Price < 50 * 10 ** 6]
df = df[df.Price > 1 * 10 ** 6]
# обрезаем пару выросов по Площади
df = df[df.Area < 1000]

# с распределением остальныз признаков все хорошо
df.shape

(52600, 11)

In [9]:
# сделаем проверку на существование станции метро
# некоторые запарсились криво
# а также присоединим информациб по станции метро
list_stations = ['Ховрино', 'Сокол', 'Остров мечты', 'Минская', 'Коммунарка', 'Бульвар Генерала Карбышева', 'Удельная', 'Ольховая', 'Орехово', 'Улица Горчакова', 'Улица Академика Королёва', 'Подрезково', 'Кратово', 'Гражданская', 'Коптево', 'Пенягино', 'Железнодорожная', 'Верхние Лихоборы', 'Москва-Товарная', 'Яхромская', 'Сухаревская', 'Фрунзенская', 'Люберцы', 'Проспект Вернадского', 'Беломорская', 'Речной вокзал', 'Одинцово', 'Улица 1905 года', 'Лесной Городок', 'Марк', 'Академическая', 'Жулебино', 'Плющево', 'Силикатная', 'Троицк', 'Лубянка', 'Марьина Роща', 'Ботанический сад', 'Десна', 'Медведково', 'Новодачная', 'Полежаевская', 'Фили', 'Университет', 'Сходня', 'Цветной бульвар', 'Тимирязевская', 'Сколково', 'Крылатское', 'Комсомольская', 'Парк Победы', 'Дубровка', 'Севастопольская', 'Домодедовская', 'Тульская', 'Угрешская', 'Зорге', 'Кусково', 'Баковка', 'Серпуховская', 'Воронцовская', 'Кутузовская', 'Химки', 'Грачёвская', 'Дегунино', 'Воробьёвы горы', 'Перерва', 'Летово', 'Каспийская', 'Беляево', 'Гольяново', 'Лобня', 'Отрадное', 'Ростокино', 'Курская', 'Боровское шоссе', 'Тропарёво', 'Алма-Атинская', 'Бутово', 'Черкизовская', 'Озёрная', 'Матвеевская', 'Студенческая', 'Прокшино', 'Есенинская', 'Опалиха', 'Фонвизинская', 'Мещерская', 'Реутов', 'Бунинская аллея', 'Бульвар Рокоссовского', 'Профсоюзная', 'Шелепиха', 'Кавказский бульвар', 'Каширская', 'Лианозово', 'Суворовская', 'Перово', 'Трубная', 'Соколиная гора', 'Третьяковская', 'Новоподрезково', 'Подольск', 'Говорово', 'Аннино', 'Нахимовский проспект', 'Кузнецкий мост', 'Лухмановская', 'Верхние котлы', 'Немчиновка', 'Боровицкая', 'Улица Академика Янгеля', 'Улица Старокачаловская', 'Деловой центр', 'Хорошёвская', 'Митино', 'Рижская', 'Мякинино', 'Зеленоград — Крюково', 'Первомайская', 'Чеховская', 'Щёлковская', 'Остафьево', 'Южный порт', 'Марксистская', 'Ватутинки', 'Локомотив', 'Ильинская', 'Тютчевская', 'Кокошкино', 'Никольское', 'Терехово ', 'Тургеневская', 'Депо', 'Нагорная', 'Лихоборы', 'Народное Ополчение', 'Бибирево', 'Быково', 'Зюзино', 'Нахабино', 'Щукинская', 'Октябрьское поле', 'Липецкая', 'Останкино', 'Новоясеневская', 'Ясенево', 'Звенигородская', 'Филатов Луг', 'Петровский Парк', 'Кантемировская', 'Новокузнецкая', 'Серп и Молот', 'Волгоградский проспект', 'Шереметьевская', 'Панки', 'Долгопрудная', 'Войковская', 'Новопеределкино', 'Ипподром', 'Китай-город', 'Люблино', 'Сортировочная', 'Окружная', 'Рассказовка', 'Южная', 'Спортивная', 'Серебряный Бор', 'Рязанский проспект', 'Вешняки', 'Петровско-Разумовская', 'Нагатинская', 'Киевская', 'Динамо', 'Толстопальцево', 'Томилино', 'Площадь Ильича', 'Потапово', 'Кропоткинская', 'Тушинская', 'Вавиловская', 'Славянский бульвар', 'Левобережная', 'Очаково', 'Сетунь', 'Раменское', 'Красный Строитель', 'Мнёвники', 'Смоленская', 'Белокаменная', 'Измайлово', 'Телецентр', 'Фирсановская', 'Кузьминки', 'Октябрьская', 'Мичуринец', 'Красные ворота', 'Новаторская', 'Щербинка', 'Пушкинская', 'Балтийская', 'Битцевский парк', 'Проспект Мира', 'Сходненская', 'Кленовый бульвар', 'ВДНХ', 'Чистые пруды', 'Покровское', 'Победа', 'Менделеевская', 'Отдых', 'Измайловская', 'Партизанская', 'Лермонтовский проспект', 'Площадь Гагарина', 'Алтуфьево', 'Владыкино', 'Ольгино', 'Кучино', 'Технопарк', 'Бирюлёво', 'Физтех', 'Битца', 'Бачуринская', 'Аэропорт', 'Бульвар Дмитрия Донского', 'Университет Дружбы Народов', 'Свиблово', 'Шаболовская', 'ЗИЛ', 'Беговая', 'Павшино', 'Кожуховская', 'Молжаниново', 'Охотный ряд', 'Улица Дмитриевского', 'Площадь трёх вокзалов', 'Ухтомская', 'Москворечье', 'Шипиловская', 'Лебедянская', 'Пролетарская', 'Новогиреево', 'Сокольники', 'Панфиловская', 'Краснопресненская', 'Театральная', 'Селигерская', 'Некрасовка', 'Трикотажная', 'Чухлинка', 'Таганская', 'Достоевская', 'Дмитровская', 'Маяковская', 'Тёплый Стан', 'Солнцево', 'Апрелевка', 'Савёловская', 'Аникеевка', 'Липовая Роща', 'Новокосино', 'Авиамоторная', 'Автозаводская', 'Москва-Сити', 'Курьяново', 'Рабочий посёлок', 'Волжская', 'Котельники', 'Каховская', 'Юго-Восточная', 'Римская', 'Водники', 'Волоколамская', 'Печатники', 'Стрешнево', 'Саларьево', 'Бутырская', 'Калужская', 'Пятницкое шоссе', 'Павелецкая', 'Варшавская', 'Солнечная', 'Румянцево', 'Улица Милашенкова', 'Фабричная', 'Поклонная', 'Хлебниково', 'Улица Сергея Эйзенштейна', 'Коньково', 'Красногвардейская', 'Лужники', 'Братиславская', 'Бульвар Адмирала Ушакова', 'Карамышевская', 'Калитники', 'Корниловская', 'Площадь Революции', 'Красногорская', 'Пионерская', 'Малаховка', 'Окская', 'Зябликово', 'Новослободская', 'Парк Культуры', 'Пражская', 'Бабушкинская', 'Добрынинская', 'Сретенский бульвар', 'Крестьянская застава', 'Чертановская', 'Пыхтино', 'ЦСКА', 'Косино', 'Красносельская', 'Баррикадная', 'Санино', 'Молодёжная', 'Давыдково', 'Крёкшино', 'Арбатская', 'Улица Скобелевская', 'Нижегородская', 'Чкаловская', 'Полянка', 'Текстильщики', 'Водный стадион', 'Генерала Тюленева', 'Выставочный центр', 'Аминьевская', 'Филёвский парк', 'Хорошёво', 'Библиотека им. Ленина', 'Выхино', 'Лефортово', 'Шоссе Энтузиастов', 'Новохохловская', 'Спартак', 'Крымская', 'Тверская', 'Моссельмаш', 'Кедровая', 'Электрозаводская', 'Алексеевская', 'Бескудниково', 'Кунцевская', 'Бауманская', 'Юго-Западная', 'Аэропорт Внуково', 'Белорусская', 'Красный Балтиец', 'Переделкино', 'Ленинский проспект', 'Салтыковская', 'Борисово', 'Красково', 'Александровский сад', 'Строгино', 'Новые Черёмушки', 'Царицыно', 'Марьино', 'Мичуринский проспект', 'Нагатинский Затон', 'Багратионовская', 'Стахановская', 'Андроновка', 'Преображенская площадь', 'Лесопарковая', 'Внуково', 'Ломоносовский проспект', 'Планерная', 'Семёновская', 'Сосенки', 'Раменки', 'Коломенская']
list_stations = [station.lower().replace(' ', '') for station in list_stations]
df = df[df['Metro station'].isin(list_stations)]
metro = pd.read_csv('metro22222.csv', sep = ';')
metro.columns = ['Metro station', 'lat_metro', 'lon_metro']
metro['Metro station'] = metro['Metro station'].str.lower()
metro['Metro station'] = metro['Metro station'].str.replace(' ', '')
metro = metro.drop_duplicates(subset = 'Metro station')
df = pd.merge(df, metro, on = 'Metro station', how = 'left')
              
df = df.dropna(subset = 'lat_metro')
df

,Metro station,lon,lat,Number of floors,type,Number of rooms,id,Area,floor,Price,date,lat_metro,lon_metro
3,кокошкино,37.168891,55.587372,9,monolith,0.0,291213865,25.91,8,6214704.0,2024-04-30,55.599722,37.171389
4,кокошкино,37.167660,55.603110,16,monolithBrick,4.0,270537621,132.00,4,18600000.0,2024-04-30,55.599722,37.171389
5,кокошкино,37.169340,55.586364,5,monolith,1.0,296654119,40.79,1,8492621.0,2024-04-30,55.599722,37.171389
6,кокошкино,37.164157,55.588420,14,monolithBrick,0.0,300868499,24.00,10,6750000.0,2024-04-30,55.599722,37.171389
7,кокошкино,37.169340,55.586364,5,monolith,0.0,300305629,25.92,4,6533796.0,2024-04-30,55.599722,37.171389
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44880,киевская,37.576313,55.741029,9,brick,2.0,300324717,51.00,2,31000000.0,2024-05-01,55.743117,37.564132
44881,спортивная,37.576879,55.718633,16,brick,2.0,301216303,57.00,3,39000000.0,2024-05-01,55.722388,37.562041
44882,смоленская,37.576187,55.744059,9,monolith,2.0,300440425,57.00,8,39000000.0,2024-05-01,55.747713,37.583802
44883,киевская,37.574489,55.739843,10,brick,3.0,300774306,61.40,2,40000000.0,2024-05-01,55.743117,37.564132


In [84]:
# Добавим информацию по расстоянию до станции метро и расстояниб ло разных точек в центре
def calculate_distance(row):
    point_x = (row['lat'], row['lon'])
    point_y = (row['lat_metro'], row['lon_metro'])
    return haversine(point_x, point_y, unit='km')
df['distance_metro'] = df.apply(calculate_distance, axis = 1)
def calculate_distance_to_red_square(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.753544, 37.621202)
    return haversine(point_x, point_y, unit='km')
df['distance_to_red_square'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_moscow_city(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.74900, 37.53964)
    return haversine(point_x, point_y, unit='km')
df['distance_to_moscow_city'] = df.apply(calculate_distance_to_moscow_city, axis=1)

def calculate_distance_to_vdnh(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.831388, 37.629277)
    return haversine(point_x, point_y, unit='km')
df['distance_to_vdnh'] = df.apply(calculate_distance_to_vdnh, axis=1)

def calculate_distance_to_svo(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.968071, 37.434100)
    return haversine(point_x, point_y, unit='km')
df['distance_to_svo'] = df.apply(calculate_distance_to_svo, axis=1)

In [86]:
# Добавим информацию по принадлежности районам
districts = pd.read_excel('metro_info.xlsx')
districts['Metro station'] = districts['Metro station'].str.lower()
districts['Metro station'] = districts['Metro station'].str.replace(' ', '')
districts = districts.drop_duplicates(subset=['Metro station'])
df = pd.merge(df, districts, on='Metro station', how='left')

In [87]:
df.to_csv('The_full_info_cian.csv')